# Garmin Multiple Activities
A Jupyter notebook that derives gps data from the Garmin cloud an displays multible tracks.

This notebook has been created with the help of this tutorial: https://openbase.com/python/garminconnect

In [1]:
import folium
import gpxpy
import getpass

from garminconnect import (
    Garmin,
    GarminConnectConnectionError,
    GarminConnectTooManyRequestsError,
    GarminConnectAuthenticationError,
)


In [2]:
email    = input("Email address: ")
password = getpass.getpass('Password:')

try:
    # API

    ## Initialize Garmin api with your credentials
    api = Garmin(email, password)

    ## Login to Garmin Connect portal
    api.login()
except (
        GarminConnectConnectionError,
        GarminConnectAuthenticationError,
        GarminConnectTooManyRequestsError,
    ) as err:
    print("Error occurred during Garmin Connect communication: %s", err)

Email address: Josef.Krammer@gmx.net
Password:········


In [3]:
Lazarote_ids = [7973508137, 7969133715, 7964207375, 7959829169, 7955041741, 7951113112, 7946218832]


In [4]:
"""
Download Activities of a id-list and store in Files
"""
activity_ids = Lazarote_ids
gpx_data_sets = []
try:
    for activity_id in activity_ids:
        #activity_id = activity["activityId"]
        print("client.download_activities(%s)", activity_id)
        print("----------------------------------------------------------------------------------------")

        gpx_data_sets.append(api.download_activity(activity_id, dl_fmt=api.ActivityDownloadFormat.GPX))
except (
    GarminConnectConnectionError,
    GarminConnectAuthenticationError,
    GarminConnectTooManyRequestsError,
) as err:
    print("Error occurred during Garmin Connect Client get activity data: %s" % err)
    quit()
except Exception:  # pylint: disable=broad-except
    print("Unknown error occurred during Garmin Connect Client get activity data")
    quit()


client.download_activities(%s) 7973508137
----------------------------------------------------------------------------------------
client.download_activities(%s) 7969133715
----------------------------------------------------------------------------------------
client.download_activities(%s) 7964207375
----------------------------------------------------------------------------------------
client.download_activities(%s) 7959829169
----------------------------------------------------------------------------------------
client.download_activities(%s) 7955041741
----------------------------------------------------------------------------------------
client.download_activities(%s) 7951113112
----------------------------------------------------------------------------------------
client.download_activities(%s) 7946218832
----------------------------------------------------------------------------------------


In [5]:

def overlayGPXset(gpxDataSet):
    '''
    overlay a gpx route on top of an OSM map using Folium
    some portions of this function were adapted
    from this post: https://stackoverflow.com/questions/54455657/
    how-can-i-plot-a-map-using-latitude-and-longitude-data-in-python-highlight-few
    '''
    lat_min =  100
    lat_max = -100
    lon_min =  360
    lon_max = -360
    first = True
    #print(len(gpxDataSet))
    for gpxData in gpxDataSet:
        #print(first)
        points = []
        gpx = gpxpy.parse(gpxData)
        for track in gpx.tracks:
            #print('track')
            for segment in track.segments:        
                for point in segment.points:
                    points.append(tuple([point.latitude, point.longitude]))
        lat_min   = min(lat_min, min(p[0] for p in points))
        lat_max   = max(lat_max, max(p[0] for p in points))
        lon_min   = min(lon_min, min(p[1] for p in points))
        lon_max   = max(lon_max, max(p[1] for p in points))
        latitude  = sum(p[0] for p in points)/len(points)
        longitude = sum(p[1] for p in points)/len(points)
        if first:
            #print('first')
            first = False
            myMap = folium.Map(location=[latitude,longitude],zoom_start=14)
        folium.PolyLine(points, color="red", weight=2.5, opacity=1).add_to(myMap)
    #print(lat_min, lat_max, lon_min, lon_max)
    southwest = [lat_min, lon_min]
    northeast = [lat_max, lon_max]
    # fit the map to the bounding box
    myMap.fit_bounds([southwest, northeast])

    return (myMap)


In [10]:
myMap = overlayGPXset(gpx_data_sets)
myMap

Unfortunately github does not render HTML files. But you can see the result    
here: https://github.com/Sepp28/GarminMultipleActivities/blob/master/GarminMultipleActivities.ipynb    
or here: https://github.com/Sepp28/GarminMultipleActivities/LanzaroteActivityMap.html   
Running the notebook on your local machine, you can also save the map in a local html file.

In [9]:
myMap.save('LanzaroteActivityMap.html')

In [7]:
# log out of Garmin
api.logout()